# Comparison between twostep and fullstep prediction performance
In this notebook, we observe how the predictions are computed for a given learner. We also can see that the performances of the two predictions are similar, but the twostep bayesian network perform much better.

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac

import pyAgrum as gum
from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
from math import floor
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
import tqdm
import time
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold

# we define the KCs

KC_A = KnowledgeComponent(55365, "A")
KC_B = KnowledgeComponent(55363, "B")
KC_C = KnowledgeComponent(55364, "C")
KC_D = KnowledgeComponent(50988, "D")
KC_E = KnowledgeComponent(50989, "E")

A_2_C = Link(source=KC_A, target=KC_C)
B_2_C = Link(source=KC_B, target=KC_C)
C_2_D = Link(source=KC_C, target=KC_D)
C_2_E = Link(source=KC_C, target=KC_E)
domain = Domain([KC_A, KC_B, KC_C, KC_D, KC_E], [A_2_C, B_2_C, C_2_D, C_2_E])

params = {"slip": .01, "guess":.01}

# there are 5 exercises corresponding to KC A
ex_A_1 = Exercise(237957, KC_A, "qcm", ex_content="", params=params)
ex_A_2 = Exercise(237958, KC_A, "qcm", ex_content="", params=params)
ex_A_3 = Exercise(237959, KC_A, "qcm", ex_content="", params=params)
ex_A_4 = Exercise(237960, KC_A, "qcm", ex_content="", params=params)
ex_A_5 = Exercise(237961, KC_A, "qcm", ex_content="", params=params)

# there are also 5 exercises corresponding to KC B
ex_B_1 = Exercise(237947, KC_B, "qcm", ex_content="", params=params)
ex_B_2 = Exercise(237948, KC_B, "qcm", ex_content="", params=params)
ex_B_3 = Exercise(237949, KC_B, "qcm", ex_content="", params=params)
ex_B_4 = Exercise(237950, KC_B, "qcm", ex_content="", params=params)
ex_B_5 = Exercise(237951, KC_B, "qcm", ex_content="", params=params)

ex_C_1 = Exercise(237952, KC_C, "qcm", ex_content="", params=params)
ex_C_2 = Exercise(237953, KC_C, "qcm", ex_content="", params=params)
ex_C_3 = Exercise(237954, KC_C, "qcm", ex_content="", params=params)
ex_C_4 = Exercise(237955, KC_C, "qcm", ex_content="", params=params)
ex_C_5 = Exercise(237956, KC_C, "qcm", ex_content="", params=params)

ex_D_1 = Exercise(225183, KC_D, "qcm", ex_content="", params=params)
ex_D_2 = Exercise(225184, KC_D, "qcm", ex_content="", params=params)
ex_D_3 = Exercise(225185, KC_D, "qcm", ex_content="", params=params)
ex_D_4 = Exercise(225186, KC_D, "qcm", ex_content="", params=params)
ex_D_5 = Exercise(225187, KC_D, "qcm", ex_content="", params=params)

ex_E_1 = Exercise(225165, KC_E, "qcm", ex_content="", params=params)
ex_E_2 = Exercise(225166, KC_E, "qcm", ex_content="", params=params)
ex_E_3 = Exercise(225167, KC_E, "qcm", ex_content="", params=params)
ex_E_4 = Exercise(225168, KC_E, "qcm", ex_content="", params=params)
ex_E_5 = Exercise(225169, KC_E, "qcm", ex_content="", params=params)

def get_KC_from_exercise_id(exercise_id):
    if exercise_id in range(237957, 237962):
        return KC_A
    elif exercise_id in range(237947, 237952):
        return KC_B
    elif exercise_id in range(237952, 237957):
        return KC_C
    elif exercise_id in range(225183, 225188):
        return KC_D
    else:
        return KC_E
    
import pandas as pd

df = pd.read_csv("5_KCs_example_data.csv")
    
def get_strongest_folds(full, axis="user_id", nb_folds=5):
    all_elements = full[axis].unique()

    kfold = KFold(nb_folds, shuffle=True)
    folds = []
    for i, (train, test) in enumerate(kfold.split(all_elements)):
        list_of_test_ids = []
        for element_id in test:
            list_of_test_ids += list(full.query(f'{axis} == {all_elements[element_id]}').index)
        folds.append(np.array(list_of_test_ids))
    
    return folds


folds = get_strongest_folds(df, "user_id", 2)
test_ids = folds[0]

train_ids = list(set(list(df.index.values)) - set(test_ids))

df_train = df[df.index.isin(train_ids)]
df_test = df[df.index.isin(test_ids)]

from pyBKT.models import Model

# Initialize the model with an optional seed
model = Model(seed = 42, num_fits = 1)
defaults = {'order_id': 'idx', 'skill_name': 'doc_id', 'correct': 'success'}

model.fit(data = df_train, defaults = defaults)


print(model.params())
weak_linking = {KC_A: {KC_C: 'weak'}, KC_B:{KC_C: 'weak'}, 
                  KC_C:{KC_A: 'weak', KC_B:'weak', KC_D: 'weak', KC_E:'weak'},
                  KC_D: {KC_C: 'weak'}, KC_E:{KC_C: 'weak'}}
learner_pool = LearnerPool(domain, weak_linking, 'weak')

params = model.params()

# KC 50988
learner_pool.set_learn(KC_D, 0.12576)
learner_pool.set_prior(KC_D, 0.62460)
learner_pool.set_slip(KC_D, 0.32588)
learner_pool.set_guess(KC_D, 0.11352)
learner_pool.set_forget(KC_D, 0.00000)
# KC 50989
learner_pool.set_learn(KC_E, 0.42610)
learner_pool.set_prior(KC_E, 0.01927)
learner_pool.set_slip(KC_E, 0.31817)
learner_pool.set_guess(KC_E, 0.62903)
learner_pool.set_forget(KC_E, 0.00000)
# KC 55363
learner_pool.set_learn(KC_B, 0.00208)
learner_pool.set_prior(KC_B, 0.84856)
learner_pool.set_slip(KC_B, 0.04169)
learner_pool.set_guess(KC_B, 0.75210)
learner_pool.set_forget(KC_B, 0.00000)
# KC 55364
learner_pool.set_learn(KC_C, 0.56942)
learner_pool.set_prior(KC_C, 0.91344)
learner_pool.set_slip(KC_C, 0.10810)
learner_pool.set_guess(KC_C, 0.99660)
learner_pool.set_forget(KC_C, 0.00000)
# KC 55365
learner_pool.set_learn(KC_A, 0.16388)
learner_pool.set_prior(KC_A, 0.81396)
learner_pool.set_slip(KC_A, 0.13814)
learner_pool.set_guess(KC_A, 0.46995)
learner_pool.set_forget(KC_A, 0.00000)


def mae(true_vals, pred_vals):
    """ Calculates the mean absolute error. """
    return np.mean(np.abs(true_vals - pred_vals))


                        value
skill param   class          
50988 prior   default 0.53893
      learns  default 0.14974
      guesses default 0.15037
      slips   default 0.31194
      forgets default 0.00000
50989 prior   default 0.68987
      learns  default 0.04274
      guesses default 0.44875
      slips   default 0.25401
      forgets default 0.00000
55363 prior   default 0.94364
      learns  default 0.00923
      guesses default 0.66215
      slips   default 0.05847
      forgets default 0.00000
55364 prior   default 0.97487
      learns  default 0.56882
      guesses default 0.97933
      slips   default 0.09748
      forgets default 0.00000
55365 prior   default 0.85132
      learns  default 0.71514
      guesses default 0.03979
      slips   default 0.15220
      forgets default 0.00000


In [2]:
accuracy_scores = []

expected_values = []
predicted_values = []
start = time.time()

for learner_id in df_test["user_id"].unique():
    learner = Learner(learner_id, learner_pool)
    learner_evals = df[df["user_id"] == learner_id]
    answers = [[get_KC_from_exercise_id(row["exercise_id"]), row["success"]] for i, row in learner_evals.iterrows()]
    exp, pred = learner.evaluate_model_prediction(answers)
    expected_values = np.concatenate((expected_values, exp))
    predicted_values = np.concatenate((predicted_values, pred))

try:
    print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
    print('AUC', roc_auc_score(expected_values, predicted_values))
    print('MAE', mae(expected_values, predicted_values))
except:
    print(predicted_values)
end = time.time()
print(end - start)

ACC 0.7299430879383146
AUC 0.6725093802414559
MAE 0.36784088469795645
697.6449437141418


In [3]:

accuracy_scores = []

expected_values = []
predicted_values = []
start = time.time()

for learner_id in df_test["user_id"].unique():
    learner = Learner(learner_id, learner_pool)
    learner_evals = df[df["user_id"] == learner_id]
    answers = [[get_KC_from_exercise_id(row["exercise_id"]), row["success"]] for i, row in learner_evals.iterrows()]
    n_eval = len(answers)
    floor_idx = 0
    for i in range(floor_idx, n_eval):
        y, pred = learner.predict_answers_with_partial_evidences(answers[:i+1], range(i), verbose=False)
        predicted_values.append(y[f"eval({answers[i][0].name}){i+1}"])
        expected_values.append(answers[i][1])
            
try:
    print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
    print('AUC', roc_auc_score(expected_values, predicted_values))
    print('MAE', mae(np.array(expected_values), np.array(predicted_values)))
except:
    print(predicted_values)
end = time.time()
print(end - start)

KeyboardInterrupt: 